# EEEN30131 Power System Analysis: Week 03 - Newton Raphson

***&copy; 2024 Martínez Ceseña — University of Manchester, UK***

This notebook provides an example of the applicationo of Newton's method to simulate steady-state power flows across a network, and also provides the relevant `python` code which can be used to solve the examples and create new ones.

The use of the notebooks is optional and will not be marked. That said, you are strongly encouraged to play with the tools and examples, as you can explore many different variations of the power flow formulation, which will better prepare you for the exams.

## List of contents

- [Developing the mismatch equations](#Developing-the-mismatch-equations)
- [Developing the Jacobian matrix](#Developing-the-Jacobian-matrix)
- [Calculate vector of mismatches](#Calculate-vector-of-mismatches)
- [Calculate Jacobian matrix](#Calculate-Jacobian-matrix)
- [Newton's method](#Newton's-method)
- [Case studies](#Case-studies)
  - [Three-bus network](#Three-bus-network)
  - [Two-bus network](#Two-bus-network)
  - [Four-bus network](#Four-bus-network)
  - [Three-bus radial network](#Three-bus-radial-network)
  - [Create your own example](#Create-your-own-example)

## Before we begin

This notebook builds on the materials that were presented during the last two weeks. Therefore, before we begin: 
- Make sure to review the asynchronous materials provided in blackboard for EEEN30131:
  - Week 1 - Nodal analysis 
  - Week 2 - Power Flow Formulation 
  - Week 3 - Newton Raphson
- If you have any questions, please post them in the discussion boards or, if that is not possible, send an email to alex.martinezcesena@manchester.ac.uk

If this data notebook is being used in Jupyter lite, the folders where the python code that supports this notebook are stored, have to be enabled.

In [ ]:
from pathlib import Path
if Path.cwd().drive == '':
    a_dir = Path("dir")
    a_dir.mkdir(exist_ok=True)

We will need the main tools that were developed over the last two weeks (e.g., $Y_{bus}$).

In [1]:
import cmath
import math
import numpy
from Code.Wk1_EEEN30131 import get_Ybus
from Code.Wk2_EEEN30131 import get_Bus_Type, develop_PF_Equations

The tools can now be used using the approach explained in the ***EEEN30131_Week01*** and ***EEEN30131_Week02*** notebooks. 

FOr example, consider the system presented in the figure below.

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

The information of this system can be coded as follows:

In [2]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]
Load = [
    [2, complex(1.5, 0.8)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1}
]

The $Y_{bus}$ matrix can be calculated and the power flow equations can be developed with the following two tools:

In [3]:
Ybus = get_Ybus(Connectivity, True)
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Developing the mismatch equations

Now that the power flow equations have been defined, it is time to develop the mismatch equations. However, before that, it is convenient to identify all known parameters ($P$ and $Q$) and unknown variables ($V$ and $\theta$).

In [4]:
# Using tool from week 02 to get bus types
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

def get_Imp_Unk(Bus_Type, Generator):
    ''' Get implicit equations and unknown variables '''
    Number_Buses = len(Bus_Type)
    V_All = [1 for bus in range(Number_Buses)]
    𝜃_All = [0 for bus in range(Number_Buses)]
    for gen in Generator:
        if 'V' in gen.keys():
            bus = gen['Bus']-1
            V_All[bus] = gen['V']
        if '𝜃' in gen.keys():
            bus = gen['Bus']-1
            𝜃_All[bus] = gen['𝜃']

    J_P = []  # Known P
    J_Q = []  # Known Q
    J_V = []  # Unknown V
    J_𝜃 = []  # Unknown 𝜃
    for bus in range(Number_Buses):
        if Bus_Type[bus] == 1:  # PQ
            J_P.append(bus)
            J_Q.append(bus)
            J_V.append(bus)
            J_𝜃.append(bus)
        if Bus_Type[bus] == 2:  # PV
            J_P.append(bus)
            J_𝜃.append(bus)
            V_All[bus]

    return J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All

# Calling new method
J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All = get_Imp_Unk(Bus_Type, Generator)

# Printing results
print('The implicit equations correspond to:')
for bus in J_P:
    print('P'+str(bus+1), end=' ')
for bus in J_Q:
    print('Q'+str(bus+1), end=' ')
print('\nThe unknown variables are:')
for bus in J_𝜃:
    print('𝜃'+str(bus+1), end=' ')
for bus in J_V:
    print('V'+str(bus+1), end=' ')

The implicit equations correspond to:
P2 P3 Q2 
The unknown variables are:
𝜃2 𝜃3 V2 

Our objective is now to create a `string` with the developed mismatch equations. Let us begin by creating a method that would allow us to print the equation using subscripts

In [5]:
def get_string(txt, bus1, bus2=None):
    '''Create string with subscripts'''
    SUB = str.maketrans("0123456789", "₀₁₂₃₄₅₆₇₈₉")
    str1 = str(bus1+1).translate(SUB)
    if bus2 is None:
        str2 = ''
    else:
        str2 = ',' + str(bus2+1).translate(SUB)

    return txt+str1+str2

It is time to develop a couple of method to print the equations. We begin with a method (`develop_ΔPQ`) that handles the addition of signs ($+$ or $-$) between the componentt of the equation, and a second method (`develop_ΔPQ_string`) to handle brackets, trigonometric functions, etc.

In [6]:
def develop_ΔPQ(Dt_Raw):
    Dt = Dt_Raw[0]
    PQ = Dt_Raw[1]
    bus = Dt_Raw[2]
    txt = Dt_Raw[3]

    print('Δ%s = ' % get_string(txt, bus), end='')
    flg = False
    for part in range(len(Dt)):
        if Dt[part]['val1'] > 0 and flg:
            print(' + ', end='')
        flg = True
        develop_ΔPQ_string(Dt[part])

    if PQ is None:
        print(' - %s', get_string(txt, bus))
    else:
        if PQ < 0:
            print(' + %.4f' % (-1*PQ))
        else:
            print(' - %.4f' % PQ)

def develop_ΔPQ_string(Dt):
    Str = ''
    if Dt['V1'] is not None:
        Str += get_string('V', Dt['V1'])
    if Dt['V2'] is not None:
        if Dt['V1'] == Dt['V2']:
            Str += '\u00b2'
        else:
            Str += get_string('V', Dt['V2'])

    StrA = ''
    if Dt['𝜃A1'] is not None:
        StrA = Dt['scA']
        if Dt['𝜃A2'] is None:
            StrA += get_string('𝜃', Dt['𝜃A1'])
        else:
            StrA += '(' + get_string('𝜃', Dt['𝜃A1']) + '-' + \
                get_string('𝜃', Dt['𝜃A2']) + ')'

    StrB = ''
    if Dt['𝜃B1'] is not None:
        StrB = Dt['scB']
        if Dt['𝜃B2'] is None:
            StrB += get_string('𝜃', Dt['𝜃B1'])
        else:
            StrB += '(' + get_string('𝜃', Dt['𝜃B1']) + '-' + \
                get_string('𝜃', Dt['𝜃B2']) + ')'

    if Dt['val2'] is None:
        print('%.4f%s' % (Dt['val1'], Str+StrA), end='')
    else:
        if Dt['val3'] > 0:
            aux = '+'
        else:
            aux = ''
        print('%.4f%s[%.4f%s%s%.4f%s]' % (Dt['val1'], Str, Dt['val2'], StrA,
                                          aux, Dt['val3'], StrB), end='')

The mismatch equations can now be printed by calling the `develop_ΔPQ` method with the implicit equations (in `J_P` and `J_Q`).

In [7]:
for bus in J_P:
    develop_ΔPQ(P_Data[bus])

for bus in J_Q:
    develop_ΔPQ(Q_Data[bus])

ΔP₂ = 10.0000V₂sin𝜃₂ + 4.0000V₂sin(𝜃₂-𝜃₃) + 1.5000
ΔP₃ = 5.0000sin𝜃₃ + 4.0000V₂sin(𝜃₃-𝜃₂) - 1.0000
ΔQ₂ = -10.0000V₂cos𝜃₂ + 14.0000V₂²-4.0000V₂cos(𝜃₂-𝜃₃) + 0.8000


[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Developing the Jacobian matrix

The Jacobian matrix can be developed by differentiating the implicit equations with respect to the unknown variables. We already have the equations and variables, so it is time to develop a method to differentiate the equations.

Considering the characteristics of the power flow equations, it is convenient to develop a dedicated method to differentiate the sine and cosine functions.

In [8]:
def diff_sin_cos(Dt_sc, Dt_𝜃1, Dt_𝜃2, 𝜃):
    '''Developing the differentials of sine and cosine'''
    Str = ''
    Val = 1
    if Dt_𝜃1 is not None:
        if Dt_sc == 'sin':
            Str = 'cos'
        if Dt_sc == 'cos':
            Str = 'sin'
            Val = -1
        if Dt_𝜃2 is None:
            Str += get_string('𝜃', Dt_𝜃1)
        else:
            Str += '(' + get_string('𝜃', Dt_𝜃1) + '-' + \
                get_string('𝜃', Dt_𝜃2) + ')'
            if 𝜃 == Dt_𝜃2:
                Val *= -1

    return Str, Val

A method to write the string for the differential of implicit equations with respect to $\theta$ can be created with the following code.

In [9]:
def develop_J_𝜃(Dt_Raw, 𝜃):
    '''Develop equations ∂P/Q with respect to 𝜃'''
    Dt = Dt_Raw[0]
    # PQ = Dt_Raw[1]
    bus = Dt_Raw[2]
    txt = Dt_Raw[3]

    print('∂Δ%s/∂𝜃%d = ' % (get_string(txt, bus), 𝜃+1), end='')
    flg = False
    for part in range(len(Dt)):
        flg = develop_ΔPQ_𝜃(Dt[part], 𝜃, flg)
    if not flg:
        print('0')
    else:
        print()

def develop_ΔPQ_𝜃(Dt, 𝜃, flg):
    ''' String for differential of P/Q with respect to 𝜃'''
    if 𝜃 != Dt['𝜃A1'] and 𝜃 != Dt['𝜃A2'] and 𝜃 != Dt['𝜃B1'] and 𝜃 != Dt['𝜃B2']:
        return flg

    Str = ''
    if Dt['V1'] is not None:
        Str += get_string('V', Dt['V1'])
    if Dt['V2'] is not None:
        if Dt['V1'] == Dt['V2']:
            Str += '\u00b2'
        else:
            Str += get_string('V', Dt['V2'])

    StrA, ValA = diff_sin_cos(Dt['scA'], Dt['𝜃A1'], Dt['𝜃A2'], 𝜃)

    StrB, ValB = diff_sin_cos(Dt['scB'], Dt['𝜃B1'], Dt['𝜃B2'], 𝜃)

    if Dt['val2'] is None:
        if Dt['val1']*ValA > 0 and flg:
            print(' + ', end='')

        print('%.4f%s' % (Dt['val1']*ValA, Str+StrA), end='')
    else:
        if Dt['val1'] > 0 and flg:
            print(' + ', end='')
        if Dt['val3'] > 0:
            aux = '+'
        else:
            aux = ''
        print('%.4f%s[%.4f%s%s%.4f%s]'
              % (Dt['val1'], Str, Dt['val2']*ValA, StrA, aux, Dt['val3']*ValB,
                 StrB), end='')
    return True

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

A method to write the string for the differential of implicit equations with respect to $V$ can be created with the following code.

In [10]:
def develop_ΔPQ_V(Dt, V, flg):
    ''' Differential of P/Q with respect to V'''
    if V != Dt['V1'] and V != Dt['V2']:
        return flg

    Str = ''
    Val = 1
    if Dt['V1'] is not None:
        if V != Dt['V1']:
            Str += get_string('V', Dt['V1'])
    if Dt['V2'] is not None:
        if Dt['V1'] == Dt['V2']:
            Val = 2
        elif V != Dt['V2']:
            Str += get_string('V', Dt['V2'])

    StrA = ''
    if Dt['𝜃A1'] is not None:
        StrA = Dt['scA']
        if Dt['𝜃A2'] is None:
            StrA += get_string('𝜃', Dt['𝜃A1'])
        else:
            StrA += '(' + get_string('𝜃', Dt['𝜃A1']) + '-' + \
                get_string('𝜃', Dt['𝜃A2']) + ')'

    StrB = ''
    if Dt['𝜃B1'] is not None:
        StrA = Dt['scB']
        if Dt['𝜃B2'] is None:
            StrB += get_string('𝜃', Dt['𝜃B1'])
        else:
            StrB += '(' + get_string('𝜃', Dt['𝜃B1']) + '-' + \
                get_string('𝜃', Dt['𝜃B2']) + ')'

    if Dt['val1']*Val > 0 and flg:
        print(' + ', end='')

    if Dt['val2'] is None:
        print('%.4f%s' % (Dt['val1']*Val, Str+StrA), end='')
    else:
        if Dt['val3'] > 0:
            aux = '+'
        else:
            aux = ''
        print('%.4f%s[%.4f%s%s%.4f%s]' % (Dt['val1']*Val, Str, Dt['val2'],
                                          StrA, aux, Dt['val3'], StrB), end='')

    return True
    
def develop_J_V(Dt_Raw, V):
    Dt = Dt_Raw[0]
    # PQ = Dt_Raw[1]
    bus = Dt_Raw[2]
    txt = Dt_Raw[3]

    print('∂Δ%s/∂V%d = ' % (get_string(txt, bus), V+1), end='')
    flg = False
    for part in range(len(Dt)):
        flg = develop_ΔPQ_V(Dt[part], V, flg)
    if not flg:
        print('0')
    else:
        print()

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

The new methods can be used to print the developed equations that form the Jacobian matrix:

In [11]:
for bus in J_P:
    for 𝜃 in J_𝜃:
        develop_J_𝜃(P_Data[bus], 𝜃)
    for 𝜃 in J_V:
        develop_J_V(P_Data[bus], 𝜃)
    print()

for bus in J_Q:
    for 𝜃 in J_𝜃:
        develop_J_𝜃(Q_Data[bus], 𝜃)
    for 𝜃 in J_V:
        develop_J_V(Q_Data[bus], 𝜃)
    print()

∂ΔP₂/∂𝜃2 = 10.0000V₂cos𝜃₂ + 4.0000V₂cos(𝜃₂-𝜃₃)
∂ΔP₂/∂𝜃3 = -4.0000V₂cos(𝜃₂-𝜃₃)
∂ΔP₂/∂V2 = 10.0000sin𝜃₂ + 4.0000sin(𝜃₂-𝜃₃)

∂ΔP₃/∂𝜃2 = -4.0000V₂cos(𝜃₃-𝜃₂)
∂ΔP₃/∂𝜃3 = 5.0000cos𝜃₃ + 4.0000V₂cos(𝜃₃-𝜃₂)
∂ΔP₃/∂V2 = 4.0000sin(𝜃₃-𝜃₂)

∂ΔQ₂/∂𝜃2 = 10.0000V₂sin𝜃₂ + 4.0000V₂sin(𝜃₂-𝜃₃)
∂ΔQ₂/∂𝜃3 = -4.0000V₂sin(𝜃₂-𝜃₃)
∂ΔQ₂/∂V2 = -10.0000cos𝜃₂ + 28.0000-4.0000cos(𝜃₂-𝜃₃)



For the sake of convenience, we can create a method with all the code needed to display the implicit equations, unknown variables and developed jacobian matrix equations.

In [12]:
def develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator):
    J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All = get_Imp_Unk(Bus_Type, Generator)

    print('The implicit equations correspond to:')
    for bus in J_P:
        print('P'+str(bus+1), end=' ')
    for bus in J_Q:
        print('Q'+str(bus+1), end=' ')

    print('\nThe unknown variables are:')
    for bus in J_𝜃:
        print('𝜃'+str(bus+1), end=' ')
    for bus in J_V:
        print('V'+str(bus+1), end=' ')

    print('\n\nVector of missmatches')
    for bus in J_P:
        develop_ΔPQ(P_Data[bus])
    for bus in J_Q:
        develop_ΔPQ(Q_Data[bus])

    print('\nEquations that form the Jacobian matrix')
    for bus in J_P:
        for 𝜃 in J_𝜃:
            develop_J_𝜃(P_Data[bus], 𝜃)
        for 𝜃 in J_V:
            develop_J_V(P_Data[bus], 𝜃)
        print()
    for bus in J_Q:
        for 𝜃 in J_𝜃:
            develop_J_𝜃(Q_Data[bus], 𝜃)
        for 𝜃 in J_V:
            develop_J_V(Q_Data[bus], 𝜃)
        print()
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)

The implicit equations correspond to:
P2 P3 Q2 
The unknown variables are:
𝜃2 𝜃3 V2 

Vector of missmatches
ΔP₂ = 10.0000V₂sin𝜃₂ + 4.0000V₂sin(𝜃₂-𝜃₃) + 1.5000
ΔP₃ = 5.0000sin𝜃₃ + 4.0000V₂sin(𝜃₃-𝜃₂) - 1.0000
ΔQ₂ = -10.0000V₂cos𝜃₂ + 14.0000V₂²-4.0000V₂cos(𝜃₂-𝜃₃) + 0.8000

Equations that form the Jacobian matrix
∂ΔP₂/∂𝜃2 = 10.0000V₂cos𝜃₂ + 4.0000V₂cos(𝜃₂-𝜃₃)
∂ΔP₂/∂𝜃3 = -4.0000V₂cos(𝜃₂-𝜃₃)
∂ΔP₂/∂V2 = 10.0000sin𝜃₂ + 4.0000sin(𝜃₂-𝜃₃)

∂ΔP₃/∂𝜃2 = -4.0000V₂cos(𝜃₃-𝜃₂)
∂ΔP₃/∂𝜃3 = 5.0000cos𝜃₃ + 4.0000V₂cos(𝜃₃-𝜃₂)
∂ΔP₃/∂V2 = 4.0000sin(𝜃₃-𝜃₂)

∂ΔQ₂/∂𝜃2 = 10.0000V₂sin𝜃₂ + 4.0000V₂sin(𝜃₂-𝜃₃)
∂ΔQ₂/∂𝜃3 = -4.0000V₂sin(𝜃₂-𝜃₃)
∂ΔQ₂/∂V2 = -10.0000cos𝜃₂ + 28.0000-4.0000cos(𝜃₂-𝜃₃)



So far, we have produced tools to display the equations needed to develop the vector of mismatches and Jacobian matrix. It is now time to calculate the relevant numerical values.

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Calculate vector of mismatches

To calculate the vector of mismatches,we can begin by producing a method to handle the trigonometric functions.

In [13]:
def get_sin_cos(Dt_sc, Dt_𝜃1, Dt_𝜃2, 𝜃_All):
    if Dt_sc is None:
        return 1

    if Dt_𝜃1 is not None:
        Ang = 𝜃_All[Dt_𝜃1]
    else:
        Ang = 0
    if Dt_𝜃2 is not None:
        Ang -= 𝜃_All[Dt_𝜃2]

    if Dt_sc == 'sin':
        Val = math.sin(Ang)
    else:
        Val = math.cos(Ang)

    return Val

We then develop a method to check whether we are solving for $P$ or $Q$ and another method to do the calculations.

In [14]:
def get_ΔPQ(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All):
    Δ = []

    for bus in J_P:
        Δ.append(get_Δ(P_Data[bus], V_All, 𝜃_All))

    for bus in J_Q:
        Δ.append(get_Δ(Q_Data[bus], V_All, 𝜃_All))

    return Δ

def get_Δ(Dt_Raw, V_All, 𝜃_All):
    Dta = Dt_Raw[0]
    PQ = Dt_Raw[1]
    # bus = Dt_Raw[2]
    # txt = Dt_Raw[3]

    Val = 0
    for part in range(len(Dta)):
        Dt = Dta[part]

        aux = 1
        if Dt['V1'] is not None:
            aux *= V_All[Dt['V1']]
        if Dt['V2'] is not None:
            aux *= V_All[Dt['V2']]

        if Dt['val2'] is None:
            Val += aux*Dt['val1']*get_sin_cos(Dt['scA'], Dt['𝜃A1'], Dt['𝜃A2'],
                                              𝜃_All)
        else:
            Val += aux*Dt['val1'] * \
                (Dt['val2']*get_sin_cos(Dt['scA'], Dt['𝜃A1'], Dt['𝜃A2'],
                                        𝜃_All) +
                 Dt['val3']*get_sin_cos(Dt['scB'], Dt['𝜃B1'], Dt['𝜃B2'],
                                        𝜃_All))
    Val -= PQ

    return Val

The vector of missmatches can now be calculated as follows:

In [15]:
Number_Buses = len(Bus_Type)
Δ = get_ΔPQ(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All)

print('Vector of mismatches:',Δ)

Vector of mismatches: [1.5, -1.0, 0.8]


[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Calculate Jacobian matrix

To calculate the Jacobian matrix, we begin with a method to handle the diffetentiation of the sine and cosine functions.

In [16]:
def get_diff_sin_cos(Dt_sc, Dt_𝜃1, Dt_𝜃2, 𝜃_All, 𝜃):
    if 𝜃 != Dt_𝜃1 and 𝜃 != Dt_𝜃2:
        return 0

    Val = 1
    if Dt_𝜃1 is not None:
        Ang = 𝜃_All[Dt_𝜃1]
    else:
        Ang = 0
    if Dt_𝜃2 is not None:
        Ang -= 𝜃_All[Dt_𝜃2]
        if 𝜃 == Dt_𝜃2:
            Val = -1

    if Dt_sc == 'sin':
        Val *= math.cos(Ang)
    else:
        Val *= -1*math.sin(Ang)

    return Val

The methods to differentiate the missmatch equations with respect to $\theta$ and $V$ are then coded.

In [17]:
def get_J_V(Dt_Raw, V_All, 𝜃_All, V):
    Dta = Dt_Raw[0]
    # PQ = Dt_Raw[1]
    # bus = Dt_Raw[2]
    # txt = Dt_Raw[3]

    Val = 0
    for part in range(len(Dta)):
        Dt = Dta[part]

        if Dt['V1'] == V or Dt['V2'] == V:

            aux = 1
            if Dt['V1'] is not None:
                if Dt['V1'] != V:
                    aux *= V_All[Dt['V1']]
                elif Dt['V1'] == Dt['V2']:
                    aux *= 2*V_All[Dt['V1']]

            if Dt['val2'] is None:
                Val += aux*Dt['val1']*get_sin_cos(Dt['scA'], Dt['𝜃A1'],
                                                  Dt['𝜃A2'], 𝜃_All)
            else:
                Val += aux*Dt['val1'] * \
                    (Dt['val2']*get_sin_cos(Dt['scA'], Dt['𝜃A1'], Dt['𝜃A2'],
                                            𝜃_All) +
                     Dt['val3']*get_sin_cos(Dt['scB'], Dt['𝜃B1'], Dt['𝜃B2'],
                                            𝜃_All))

    return Val

def get_J_𝜃(Dt_Raw, V_All, 𝜃_All, 𝜃):
    Dta = Dt_Raw[0]
    # PQ = Dt_Raw[1]
    # bus = Dt_Raw[2]
    # txt = Dt_Raw[3]

    Val = 0
    for part in range(len(Dta)):
        Dt = Dta[part]

        aux = 1
        if Dt['V1'] is not None:
            aux *= V_All[Dt['V1']]
        if Dt['V2'] is not None:
            aux *= V_All[Dt['V2']]

        if Dt['scB'] is None:
            Val += aux*Dt['val1']*get_diff_sin_cos(Dt['scA'], Dt['𝜃A1'],
                                                   Dt['𝜃A2'], 𝜃_All, 𝜃)
        else:
            Val += aux*Dt['val1'] * \
                (Dt['val2']*get_diff_sin_cos(Dt['scA'], Dt['𝜃A1'], Dt['𝜃A2'],
                                             𝜃_All, 𝜃) +
                 Dt['val3']*get_diff_sin_cos(Dt['scB'], Dt['𝜃B1'], Dt['𝜃B2'],
                                             𝜃_All, 𝜃))

    return Val

Finally, a method to select the proper differentials to use (i.e., $\partial \Delta P/\partial \theta$, $\partial \Delta P/\partial V$, $\partial \Delta Q/\partial \theta$ or $\partial \Delta Q/\partial V$) is produced.

In [18]:
def get_Jacobian(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All):
    Number_J = len(J_V) + len(J_𝜃)
    J = numpy.zeros((Number_J, Number_J))

    i = 0
    for bus in J_P:
        j = 0
        for 𝜃 in J_𝜃:
            J[i][j] = get_J_𝜃(P_Data[bus], V_All, 𝜃_All, 𝜃)
            j += 1
        for V in J_V:
            J[i][j] = get_J_V(P_Data[bus], V_All, 𝜃_All, V)
            j += 1
        i += 1

    for bus in J_Q:
        j = 0
        for 𝜃 in J_𝜃:
            J[i][j] = get_J_𝜃(Q_Data[bus], V_All, 𝜃_All, 𝜃)
            j += 1
        for V in J_V:
            J[i][j] = get_J_V(Q_Data[bus], V_All, 𝜃_All, V)
            j += 1
        i += 1

    return J

J=get_Jacobian(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All)
print('Jacobian:\n', J)

Jacobian:
 [[14. -4.  0.]
 [-4.  9.  0.]
 [ 0.  0. 14.]]


[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Newton's method

To complete the first iteration of Newton's method, we need to calculate the vector of corrections and use it to update the vector of unknown variables.

In [19]:
def update_Unknowns(J, Δ, J_𝜃, J_V, 𝜃_All, V_All):
    import numpy
    dx = numpy.linalg.inv(J).dot(Δ)
    Threshold = max(abs(dx))

    j = 0
    for 𝜃 in J_𝜃:
        𝜃_All[𝜃] -= dx[j]
        j += 1
    for V in J_V:
        V_All[V] -= dx[j]
        j += 1
    return V_All, 𝜃_All, dx, Threshold

V_All, 𝜃_All, dx, Threshold = update_Unknowns(J, Δ, J_𝜃, J_V, 𝜃_All, V_All)
print('Updated voltage magnitudes [pu]: ', V_All)
print('Updated voltage angles    [rad]: ', 𝜃_All)

Updated voltage magnitudes [pu]:  [1, 0.9428571428571428, 1]
Updated voltage angles    [rad]:  [0, -0.08636363636363636, 0.07272727272727272]


Newton's method can now be coded by adding the methods above within a loop, which will iteratively repeat the procedure until the results converge (i.e., error tolerance falls within a threshold). 

In [20]:
Ybus = get_Ybus(Connectivity, True)
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)
def Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator, Disp_Iter=1):
    J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All = get_Imp_Unk(Bus_Type, Generator)

    iterations = 0
    flg = True
    while flg:
        Δ = get_ΔPQ(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All)
        J = get_Jacobian(P_Data, Q_Data, J_P, J_Q, J_V, J_𝜃, V_All, 𝜃_All)
        V_All, 𝜃_All, dx, Threshold = update_Unknowns(J, Δ, J_𝜃, J_V, 𝜃_All,
                                                      V_All)

        iterations += 1
        if iterations <= Disp_Iter:
            print('ITERATION:', iterations)
            print('Vector of mismatches:', Δ)
            print('Jacobian:\n', J)
            print('dx', dx)
            print('Updated voltage magnitudes [pu]: ', V_All)
            print('Updated voltage angles    [rad]: ', 𝜃_All)
            print()

        if iterations > 20:
            print('The model failed to converge after 20 iterations')
            flg = False
            Succes = False
        if Threshold < 0.0001:
            flg = False
            Succes = True
    if Disp_Iter > 0:
        print('RESULTS:')
        print('V:', V_All, '(pu)')
        print('𝜃:', 𝜃_All, '(rad)')
        print()

    return V_All, 𝜃_All, Threshold, Succes

V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
print('Voltage magnitudes [pu]: ', V_All)
print('Voltage angles    [rad]: ', 𝜃_All)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

Calculating the voltages is generally not sufficient to assess the conditions of the power system. Thus, it is convenient to also calculate the net power injections, currents, and power flows and losses.

In [21]:
def get_Parameters(Connectivity, V_All, 𝜃_All, Succes):
    ''' Calculate additional parameters '''
    if not Succes:
        return

    Base = 100
    Number_Buses = len(V_All)

    Results = {}
    Results['Connectivity'] = Connectivity
    Results['Voltage'] = []
    Results['Current'] = []
    Results['Sending_Power'] = []
    Results['Receiving_Power'] = []
    Results['Net_Power'] = [0 for bus in range(Number_Buses)]
    Results['Loss'] = []

    # Get complex voltages
    for bus in range(Number_Buses):
        Results['Voltage'].append(V_All[bus]*complex(math.cos(𝜃_All[bus]),
                                                     math.sin(𝜃_All[bus])))

    for branch in Connectivity:
        s = branch[0]-1
        r = branch[1]-1
        Y = 1/branch[2]

        # Complex voltages
        Vs = Results['Voltage'][s]
        Vr = Results['Voltage'][r]

        # Complex current
        I = Y * (Vs - Vr)

        # Power at both ends of the line
        Ss = Vs*I.conjugate()*Base
        Sr = -Vr*I.conjugate()*Base

        # Store data
        Results['Current'].append(I)
        Results['Sending_Power'].append(Ss)
        Results['Receiving_Power'].append(Sr)
        Results['Loss'].append(Ss+Sr)
        Results['Net_Power'][s] += Ss
        Results['Net_Power'][r] += Sr

    return Results

Finally, let us develop a method to visualize all the calculated network parameters.

In [22]:
def Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes):
    Results = get_Parameters(Connectivity, V_All, 𝜃_All, Succes)
    if not Succes:
        return

    # Number_Buses = len(V_All)
    print('VOLTAGES  [pu] [deg]:')
    xn = 0
    for V in Results['Voltage']:
        xn += 1
        print('%2.0f) %8.4f +j %8.4f (%8.4f ∠ %8.4f)'
              % (xn, V.real, V.imag, abs(V), cmath.phase(V)*180/math.pi))

    print('NET POWER INJECTIONS [MVA]:')
    xn = 0
    for S in Results['Net_Power']:
        xn += 1
        print('%2.0f) %8.4f +j %8.4f' % (xn, S.real, S.imag))

    print('CURRENTS [pu] [deg]:')
    xb = 0
    for branch in Results['Connectivity']:
        s = branch[0]
        r = branch[1]
        I = Results['Current'][xb]
        xb += 1
        print('%2.0f-%2.0f) %8.4f +j %8.4f (%8.4f ∠ %8.4f)'
              % (s, r, I.real, I.imag, abs(I), cmath.phase(I)*180/math.pi))

    print('POWER FLOWS [MVA]:')
    print('      From:                To:                   Loss:')
    xb = 0
    for branch in Results['Connectivity']:
        s = branch[0]
        r = branch[1]
        Ss = Results['Sending_Power'][xb]
        Sr = Results['Receiving_Power'][xb]
        xb += 1
        print('%2.0f-%2.0f) %8.4f +j %8.4f %8.4f +j %8.4f (%8.4f +j %8.4f)'
              % (s, r, Ss.real, Ss.imag, Sr.real, Sr.imag, Ss.real+Sr.real,
                 Ss.imag+Sr.imag))

Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

VOLTAGES  [pu] [deg]:
 1)   1.0000 +j   0.0000 (  1.0000 ∠   0.0000)
 2)   0.9273 +j  -0.0874 (  0.9314 ∠  -5.3840)
 3)   0.9972 +j   0.0748 (  1.0000 ∠   4.2893)
NET POWER INJECTIONS [MVA]:
 1)  50.0000 +j  74.0723
 2) -150.0000 +j -80.0000
 3) 100.0000 +j  34.1228
CURRENTS [pu] [deg]:
 1- 2)   0.8740 +j  -0.7267 (  1.1366 ∠ -39.7442)
 1- 3)  -0.3740 +j  -0.0140 (  0.3742 ∠ -177.8553)
 2- 3)  -0.6488 +j   0.2795 (  0.7064 ∠ 156.6937)
POWER FLOWS [MVA]:
      From:                To:                   Loss:
 1- 2)  87.3964 +j  72.6719 -87.3964 +j -59.7526 (  0.0000 +j  12.9193)
 1- 3) -37.3964 +j   1.4005  37.3964 +j   1.4005 (  0.0000 +j   2.8009)
 2- 3) -62.6036 +j -20.2474  62.6036 +j  32.7223 (  0.0000 +j  12.4749)


[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

## Case studies

### Three-bus network

The example presented above can be solved with the tools as follows:

<img src="Figures/Week02_3Bus.png" alt="Fig01" class="bg-primary" width="500px">

In [23]:
Connectivity = [
    [1, 2, complex(0, 0.10)],
    [1, 3, complex(0, 0.20)],
    [2, 3, complex(0, 0.25)]
]
Load = [
    [2, complex(1.5, 0.8)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1}
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 3 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
    2  |    1 -  3 | 0.2j
    3  |    2 -  3 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-15.j -0.+10.j -0. +5.j]
 [-0.+10.j  0.-14.j -0. +4.j]
 [-0. +5.j -0. +4.j  0. -9.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.8000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*10.0*sin(0-𝜃₂) + 1*1*5.0*sin(0-𝜃₃)
P₁ = -10.0000V₂sin(𝜃₂)-5.0000sin(𝜃₃).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,₃cos𝜃₂,₃+B₂,₃sin𝜃₂,₃] 
P₂ = V₂*V₁*

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Two-bus network

Let us now try a different example, a 2-bus system:

![Week02_2Bus.png](Figures/Week02_2Bus.png)

In [24]:
Connectivity = [
    [1, 2, complex(0, 0.10)]
]
Load = [
    [2, complex(0.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 1 branches and 2 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.1j
_______|___________|__________

Ybus = 
 [[ 0.-10.j -0.+10.j]
 [-0.+10.j  0.-10.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂
P₁ = 1*V₂*10.0*sin(0-𝜃₂)
P₁ = -10.0000V₂sin(𝜃₂).....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] 
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁ + V₂*V₂*B₂,₂*sin𝜃₂,₂
P₂ = V₂*V₁*B₂,₁*sin𝜃₂,₁
-0.5 = V₂*1*10.0*sin(𝜃₂-0)
-0.5 = 10.0000V₂sin(𝜃₂).....[Implicit]

Q1:
Q₁ = V₁V₁[G₁,₁sin𝜃₁,₁-B₁,₁cos𝜃₁,₁] + V₁V₂[G₁,₂sin𝜃₁,₂-B₁,₂cos𝜃₁,₂] 
Q₁ = V₁*V₁*-B₁,₁*cos𝜃₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = V₁²*-B₁,₁ + V₁*V₂*-B₁,₂*cos𝜃₁,₂
Q₁ = 1²*--

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Four-bus network

Let us now try a bigger network:

![Week02_4Bus.png](Figures/Week02_4Bus.png)

In [25]:
Connectivity = [
    [1, 2, complex(0, 0.25)],
    [1, 3, complex(0, 0.5)],
    [3, 4, complex(0, 0.25)]
]
Load = [
    [2, complex(0.5, 0.3)],
    [3, complex(1, 0.5)],
    [4, complex(1, 0.5)],
]
Generator = [
    {'Bus':1, 'V':1, '𝜃':0 },
    {'Bus':3, 'V':1, 'P':1.2 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 3 branches and 4 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | 0.25j
    2  |    1 -  3 | 0.5j
    3  |    3 -  4 | 0.25j
_______|___________|__________

Ybus = 
 [[ 0.-6.j -0.+4.j -0.+2.j  0.+0.j]
 [-0.+4.j  0.-4.j  0.+0.j  0.+0.j]
 [-0.+2.j  0.+0.j  0.-6.j -0.+4.j]
 [ 0.+0.j  0.+0.j -0.+4.j  0.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0000   0.0000         ?         ?
   2     PQ        ?        ?   -0.5000   -0.3000
   3     PV   1.0000        ?    0.2000         ?
   4     PQ        ?        ?   -1.0000   -0.5000

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] + V₁V₄[G₁,₄cos𝜃₁,₄+B₁,₄sin𝜃₁,₄] 
P₁ = V₁*V₁*B₁,₁*sin𝜃₁,₁ + V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = V₁*V₂*B₁,₂*sin𝜃₁,₂ + V₁*V₃*B₁,₃*sin𝜃₁,₃
P₁ = 1*V₂*4.0*sin(0-𝜃₂) + 1*1*2.0*sin(0-𝜃₃)
P₁ = -4.0000V₂sin(𝜃₂)-2.0000sin(𝜃₃).

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Three-bus radial network

We can also develop the equations for networks with complex impedances:

![Week02_2Bus_radial.png](Figures/Week02_2Bus_radial.png)

In [26]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)

### Create your own example

Try to create your own examples:

In [27]:
Connectivity = [
    [1, 2, complex(0.1, 0.2)],
    [2, 3, complex(0.1, 0.2)]
]
Load = [
    [2, complex(1.5, 0.5)]
]
Generator = [
    {'Bus':1, 'V':1.05, '𝜃':0 },
    {'Bus':3, 'P':1, 'V':1 }
]

# From Week 01
Ybus = get_Ybus(Connectivity, True)

# From Week 02
P_Data, Q_Data = develop_PF_Equations(Load, Generator, Ybus, True)
Bus_Data, Bus_Type = get_Bus_Type(Ybus, Load, Generator)

# From Week 03
develop_Jacobian(P_Data, Q_Data, Bus_Type, Generator)
V_All, 𝜃_All, Threshold, Succes = Newtons_Method(P_Data, Q_Data, Bus_Data, Bus_Type, Generator)
Visualize_Elec(Connectivity, V_All, 𝜃_All, Succes)

The network has 2 branches and 3 buses
______________________________
Branch | From - To | Impedance
------------------------------
    1  |    1 -  2 | (0.1+0.2j)
    2  |    2 -  3 | (0.1+0.2j)
_______|___________|__________

Ybus = 
 [[ 2.-4.j -2.+4.j  0.+0.j]
 [-2.+4.j  4.-8.j -2.+4.j]
 [ 0.+0.j -2.+4.j  2.-4.j]]

Bus:  Type:       V:       𝜃:     Pinj:     Qinj:
   1  Slack   1.0500   0.0000         ?         ?
   2     PQ        ?        ?   -1.5000   -0.5000
   3     PV   1.0000        ?    1.0000         ?

***DEVELOP AND SIMPLIFY***
P1:
P₁ = V₁V₁[G₁,₁cos𝜃₁,₁+B₁,₁sin𝜃₁,₁] + V₁V₂[G₁,₂cos𝜃₁,₂+B₁,₂sin𝜃₁,₂] + V₁V₃[G₁,₃cos𝜃₁,₃+B₁,₃sin𝜃₁,₃] 
P₁ = V₁*V₁*[G₁,₁*cos𝜃₁,₁+B₁,₁*sin𝜃₁,₁] + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = V₁²*G₁,₁*cos𝜃₁,₁ + V₁*V₂*[G₁,₂*cos𝜃₁,₂+B₁,₂*sin𝜃₁,₂]
P₁ = 1.05²*2.0*cos(0-0) + 1.05*V₂*[-2.0*cos(0-𝜃₂)+4.0*sin(0-𝜃₂)]
P₁ = 2.2050 + 1.0500V₂[-2.0000cos(𝜃₂)-4.0000sin(𝜃₂)].....[Explicit]

P2:
P₂ = V₂V₁[G₂,₁cos𝜃₂,₁+B₂,₁sin𝜃₂,₁] + V₂V₂[G₂,₂cos𝜃₂,₂+B₂,₂sin𝜃₂,₂] + V₂V₃[G₂,

[Back to top](#EEEN30131-Power-System-Analysis:-Week-03---Newton-Raphson)